In [1]:
#importy
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re

In [56]:
#tworze pusty dataframe
df = pd.DataFrame(columns=['tytul','typ','miejsce','pokoje','powierzchnia','cena'])
df

,tytul,typ,miejsce,pokoje,powierzchnia,cena


In [57]:
#link
url = 'https://www.otodom.pl/wynajem/mieszkanie/tychy/?search%5Bfilter_float_m%3Ato%5D=50&search%5Bcity_id%5D=166'
strona = requests.get(url)

In [58]:
html = strona.content

In [59]:
soup = bs(html,'lxml')

In [60]:
with open("otodom.html",'wb') as plik:
    plik.write(soup.prettify(encoding='utf-8'))

In [7]:
'''
notatki dot. tagów
- form, id="pagerForm", -> 'a',"" -> numery stron.
- <div class="offer-item-details">
    -
'''

'\nnotatki dot. tagów\n- form, id="pagerForm", -> \'a\',"" -> numery stron.\n- <div class="offer-item-details">\n    -\n'

### Tworzenie listy dostępnych stron z wynikami wyszukiwania.
I obsługi limitu wyszukiwań

In [8]:
'''
pagers = soup.find('ul',"pager")
pagers'''
#nieuzywane

'\npagers = soup.find(\'ul\',"pager")\npagers'

In [9]:
'''#znajdowanie aktualnego numeru strony
p = soup.find_all('a','active')
p
p[1].text
page_active = int(p[1].text)
page_active'''
#będzie niepotrzebne, kod poniżej tworzy listę stron.

"#znajdowanie aktualnego numeru strony\np = soup.find_all('a','active')\np\np[1].text\npage_active = int(p[1].text)\npage_active"

In [61]:
p2 = soup.find('form', {"id":'pagerForm'})
p2= p2.find_all('a',"")

strony = []

for a in p2:
    strona = a.text
    if strona != "":
        strona = int(strona)
        strony.append(strona)
    else:
        pass
        

strona_max = max(strony)
strona_min = min(strony)

In [62]:
strona_min

1

In [63]:
strona_max

2

In [64]:
limit_wyszukiwania = 0

def limit_wyszukiwan(strona_max):
    if strona_max > 50:
        limit_wyszukiwania =+ 50
    else:
        limit_wyszukiwania =+ strona_max
    print(limit_wyszukiwania)

In [65]:
limit_wyszukiwan(strona_max)

2


### Tworzenie pętli do spisania wszystkich ogłoszeń z strony

In [15]:
oferty = soup.find_all('article')

In [17]:
len(oferty)

27

In [18]:
oferty[0]

<article class="offer-item ad_id4a4Lw" data-featured-name="promo_top_ads" data-item-id="4a4Lw" data-tracking-id="61506945" data-url="https://www.otodom.pl/pl/oferta/tychy-najpiekniejsze-reprezentacyjne-mieszkanie-ID4a4Lw.html#71b39355d2" id="offer-item-ad_id4a4Lw">
<figure class="offer-item-image" data-featured-name="no_offer" data-quick-gallery='[{"photo":"https:\/\/ireland.apollo.olxcdn.com\/v1\/files\/eyJmbiI6ImV6YnEwZWF4cnhycjMtQVBMIiwidyI6W3siZm4iOiJqMWozbzEzbTZiZ24xLUFQTCIsInMiOiIxNCIsInAiOiIxMCwtMTAiLCJhIjoiMCJ9XX0.IfkLn0Sx_R55SgHJAo1Ltg3CCEwmflYm_oNq4pJwl8Q\/image;s=655x491;q=80","thumb":"https:\/\/ireland.apollo.olxcdn.com\/v1\/files\/eyJmbiI6ImV6YnEwZWF4cnhycjMtQVBMIiwidyI6W3siZm4iOiJqMWozbzEzbTZiZ24xLUFQTCIsInMiOiIxNCIsInAiOiIxMCwtMTAiLCJhIjoiMCJ9XX0.IfkLn0Sx_R55SgHJAo1Ltg3CCEwmflYm_oNq4pJwl8Q\/image;s=184x138;q=80"},{"photo":"https:\/\/ireland.apollo.olxcdn.com\/v1\/files\/eyJmbiI6InpucXJtdjdnanBrbDEtQVBMIiwidyI6W3siZm4iOiJqMWozbzEzbTZiZ24xLUFQTCIsInMiOiIxNCIsInAiOiIxMCwtMT

In [19]:
tytul = oferty[0].find('span', {"class":"offer-item-title"}).text
tytul

'TYCHY - najpiękniejsze, reprezentacyjne mieszkanie'

In [20]:
typ = oferty[0].find('span', {"class": "hidden-xs"}).text.strip().rstrip(":")
typ

'Mieszkanie na wynajem'

In [21]:
miejsce = oferty[0].find('p', {"class": "text-nowrap"}).text
miejsce = miejsce[miejsce.find(":",0,-1)+2:]
miejsce

'Tychy, śląskie'

In [22]:
pokoje = int(oferty[0].find('li', {'class': 'offer-item-rooms hidden-xs'}).text[0])
pokoje

2

In [23]:
powierzchnia = int(re.findall(r'\d+',oferty[0].find('li', {"class": "hidden-xs offer-item-area"}).text)[0])
powierzchnia

42

In [24]:
#cena = oferty[0].find('li', {'class': "offer-item-price"}).text.strip()
cena = int(re.findall(r'\d+',re.sub(r'\s+','',oferty[0].find('li', {'class': "offer-item-price"}).text.strip()))[0])
cena

2200

In [30]:
#wynajmujący
wynajmujacy = oferty[0].find('ul', {'class': "params-small clearfix hidden-xs"}).text.strip()
wynajmujacy

'Oferta prywatna'

In [90]:
#definiowanie pętli do sprawdzania parametrów
def parametry():
    
    
    l_tytul = []
    l_typ = []
    l_miejsce = []
    l_pokoje = []
    l_powierzchnia = []
    l_cena = []
    l_wynajmujacy = []
    ##lista_list = [l_tytul,l_typ,l_miejsce,l_pokoje,l_powierzchnia,l_cena,l_wynajmujacy]
    
    oferty = soup.find_all('article')
    

    for oferta in oferty:
        
        
        tytul = oferta.find('span', {"class":"offer-item-title"}).text
        l_tytul.append(tytul)
        
        typ = oferta.find('span', {"class": "hidden-xs"}).text.strip().rstrip(":")
        l_typ.append(typ)
        
        miejsce = oferta.find('p', {"class": "text-nowrap"}).text
        miejsce = miejsce[miejsce.find(":",0,-1)+2:]
        l_miejsce.append(miejsce)
        
        pokoje = int(oferta.find('li', {'class': 'offer-item-rooms hidden-xs'}).text[0])
        l_pokoje.append(pokoje)
        
        powierzchnia = int(re.findall(r'\d+',oferta.find('li', {"class": "hidden-xs offer-item-area"}).text)[0])
        l_powierzchnia.append(powierzchnia)
        
        cena = int(re.findall(r'\d+',re.sub(r'\s+','',oferta.find('li', {'class': "offer-item-price"}).text.strip()))[0])
        l_cena.append(cena)
        
        wynajmujacy = oferta.find('ul', {'class': "params-small clearfix hidden-xs"}).text.strip()
        l_wynajmujacy.append(wynajmujacy)
    
    '''df = pd.DataFrame(columns=['tytul','typ','miejsce','pokoje','powierzchnia','cena','wynajmujacy'])
    df.append(l_tytul)'''
    
    df1 = df = pd.DataFrame({'tytul':l_tytul, 
                             "typ":l_typ,
                             "miejsce": l_miejsce,
                             "pokoje": l_pokoje,
                             "powierzchnia": l_powierzchnia,
                             "cena": l_cena,
                             "wynajmujacy": l_wynajmujacy}) 
    
    return df1
    
    
     
    

In [92]:


tabela = parametry()
tabela

,tytul,typ,miejsce,pokoje,powierzchnia,cena,wynajmujacy
0,"TYCHY - najpiękniejsze, reprezentacyjne mieszk...",Mieszkanie na wynajem,"Tychy, śląskie",2,42,2200,Oferta prywatna
1,Nowoczesne mieszkanie - Strefa Ekonomiczna Tychy,Mieszkanie na wynajem,"Tychy, śląskie",2,39,1600,Oferta prywatna
2,Wynajmę bezpośrednio,Mieszkanie na wynajem,"Tychy, śląskie",2,36,1500,Obsługa zdalna\n\n ...
3,Mieszkanie do wynajęcia ul Skalna,Mieszkanie na wynajem,"Tychy, śląskie",2,40,2000,MS HOME Skaba
4,Mieszkanie w nowym bloku z windą - czynsz w cenie,Mieszkanie na wynajem,"Tychy, Żwaków",2,41,1800,Aston Biuro Nieruchomości
5,"TYCHY - najpiękniejsze, reprezentacyjne mieszk...",Mieszkanie na wynajem,"Tychy, śląskie",2,42,2200,Oferta prywatna
6,"Wynajem mieszkania 48mkw, ul. Konecznego",Mieszkanie na wynajem,"Tychy, śląskie",2,48,1709,Asset Nieruchomości- POSZUKUJE MIESZKAŃ i DOMÓW.
7,kawalerka w Tychach,Mieszkanie na wynajem,"Tychy, śląskie",1,38,1400,Oferta prywatna
8,"Mieszkanie 42m2, 2 pokoje, dla pierwszego loka...",Mieszkanie na wynajem,"Tychy, śląskie",2,41,1700,Oferta prywatna
9,Nowoczesne mieszkanie - Strefa Ekonomiczna Tychy,Mieszkanie na wynajem,"Tychy, śląskie",2,39,1600,Oferta prywatna
